In [5]:
import requests


csv_url = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"
response = requests.get(csv_url, stream=True)

if response.status_code == 200:
    with open("data/openfoodfacts_products2.csv.gz", "wb") as file:
        for chunk in response.iter_content(chunk_size=8000):
            file.write(chunk)
    print("Télechargement du fichier réussi.")
else:
    print(f"Erreur lors du télechargement du fichier : {response.status_code}")


Télechargement du fichier réussi.


In [6]:
import pyspark
print(pyspark.__version__)

3.5.0


In [7]:
import pyspark.pandas as ps


In [8]:

df = ps.read_csv("data/openfoodfacts_products2.csv.gz", sep="\t", mode="PERMISSIVE")

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [11]:
df['nutriscore_grade'].unique()

0           unknown
1                 e
2                 d
3                 c
4    not-applicable
5                 b
6                 a
7              None
Name: nutriscore_grade, dtype: object

Identifier les collones completement ou trop vides et les supprimer 

In [4]:
seuil = 95

colonnes_completement_vides = df.isna().sum() / len(df) * 100 > seuil
colonnes_completement_vides_liste = colonnes_completement_vides[colonnes_completement_vides].index.tolist()
print(len(colonnes_completement_vides_liste))
colonnes_completement_vides_liste

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


116


['abbreviated_product_name',
 'generic_name',
 'packaging_text',
 'origins',
 'origins_tags',
 'origins_en',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'allergens_en',
 'traces',
 'no_nutrition_data',
 'additives',
 'owner',
 'data_quality_errors_tags',
 'energy-from-fat_100g',
 'butyric-acid_100g',
 'caproic-acid_100g',
 'caprylic-acid_100g',
 'capric-acid_100g',
 'lauric-acid_100g',
 'myristic-acid_100g',
 'palmitic-acid_100g',
 'stearic-acid_100g',
 'arachidic-acid_100g',
 'behenic-acid_100g',
 'lignoceric-acid_100g',
 'cerotic-acid_100g',
 'montanic-acid_100g',
 'melissic-acid_100g',
 'unsaturated-fat_100g',
 'monounsaturated-fat_100g',
 'omega-9-fat_100g',
 'polyunsaturated-fat_100g',
 'omega-3-fat_100g',
 'omega-6-fat_100g',
 'alpha-linolenic-acid_100g',
 'eicosapentaenoic-acid_100g',
 'docosahexaenoic-acid_100g',
 'linoleic-acid_100g',
 'arachidonic-acid_100g',
 'gamma-linolenic-acid_100g',
 'dihomo-gamma-linolenic-acid_100g',
 'oleic-acid_100g',
 'elaidic-acid_100

In [5]:
df_sans_colonnes_vides = df.drop(columns=colonnes_completement_vides_liste)

In [6]:
len(df_sans_colonnes_vides.columns)

93

In [17]:
df_sans_colonnes_vides.shape 

(3866756, 93)

In [19]:
df_sans_colonnes_vides.to_csv("data/off_products_no_useless_columns.csv", index=False)

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [26]:
df_avec_nutriscore = df_sans_colonnes_vides[~df_sans_colonnes_vides['nutriscore_score'].isna()]
df_avec_nutriscore.shape

(1267779, 93)

In [27]:
df_avec_nutriscore.to_csv("data/off_products_with_nutriscore.csv", index=False)

/usr/local/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
